In [1]:
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn import tree
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve,roc_auc_score,precision_score,recall_score,auc

In [4]:
dataSet = pd.read_csv('data/dataAndre.csv')
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26884 entries, 0 to 26883
Data columns (total 85 columns):
LIMIT_BAL                     26884 non-null float64
SEX                           26884 non-null float64
BILL_AMT_DEC                  26884 non-null float64
BILL_AMT_NOV                  26884 non-null float64
BILL_AMT_OCT                  26884 non-null float64
BILL_AMT_SEP                  26884 non-null float64
BILL_AMT_AUG                  26884 non-null float64
BILL_AMT_JUL                  26884 non-null float64
PAY_AMT_DEC                   26884 non-null float64
PAY_AMT_NOV                   26884 non-null float64
PAY_AMT_OCT                   26884 non-null float64
PAY_AMT_SEP                   26884 non-null float64
PAY_AMT_AUG                   26884 non-null float64
PAY_AMT_JUL                   26884 non-null float64
DEFAULT PAYMENT JAN           26884 non-null float64
SEX_MISSING                   26884 non-null float64
EDUCATION_MISSING             26884 non-nul

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataSet, test_size = 0.2)
target = 'DEFAULT PAYMENT JAN'
features = test.columns[test.columns!=target]

In [6]:
x = dataSet[features]
y = dataSet[target]
xTr = train[features]
yTr = train[target]
xTe = test[features]
yTe = test[target]

# Confusion matrix Accuracy f1

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
ada2 = AdaBoostClassifier(n_estimators=500, random_state=0)
ada2 = ada2.fit(xTr,yTr)
yPre=ada2.predict(xTe)
conf=confusion_matrix(yTe, yPre)

In [ ]:
from sklearn.metrics import f1_score
f1_score(yTe, yPre)

0.46690610569522828

In [ ]:
import utils
utils.get_best_threshold(ada2,xTe,yTe)

/Users/pietro/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [16]:
import utils
from sklearn.metrics import f1_score

params = {
    'threshold' : [0.1, 0.53, 0.8]
}

grid = utils.ThresholdGridCV(utils.ThresholdAdaBoost, params, cv=10, random_state=42, verbose=2, metric=f1_score)
grid.fit(xTr, yTr)

best = grid.best_model()
print("best score: {0}".format(grid.best_score()))

utils.plot_roc_curve(best, X_test, y_test)

Converting data in numpy array
Evaluating model with parameters: {'threshold': 0.1}
Training fold #1

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Evaluation using crossvalidation

In [ ]:
dt_entropy = tree.DecisionTreeClassifier()

In [ ]:
dt_entropy = tree.DecisionTreeClassifier('entropy',max_depth=10)
dt_entropy = dt_entropy.fit(x,y)
xval_entropy = model_selection.cross_val_score(dt_entropy, x, y, cv=KFold(n_splits=10, shuffle=True, random_state=1234))
print("Decision Tree (Entropy) Accuracy = %3.2f +/- %.4f" %(np.average(xval_entropy),np.std(xval_entropy)))

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
rf = rf.fit(x,y)
rf_eval = model_selection.cross_val_score(rf, x, y, cv=StratifiedKFold(n_splits=10,random_state=52725,shuffle=True))
print("Random Forest\t%4.3f\t%4.3f" % (np.average(rf_eval), np.std(rf_eval)))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
ada = ada.fit(x,y)
ada_eval = model_selection.cross_val_score(ada, x, y, cv=StratifiedKFold(n_splits=10,random_state=52725,shuffle=True))
print("Ada\t%4.3f\t%4.3f" % (np.average(rf_eval), np.std(rf_eval)))

## 